In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import sys
sys.path.insert(0, '../../../')

import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch

from src.difsched.config.Configs.EnvConfigs import getEnvConfig, visualizeEnvConfig
from src.difsched.env.Environment.EnvironmentSim import createEnv
from src.difsched.utils.Helpers.EnvInterface import EnvInterface

In [4]:
configIdx = 0
envParams = getEnvConfig(configIdx)
visualizeEnvConfig(envParams)
trafficDataParentPath = f'../../../data/raw/traffic'
env = createEnv(envParams, trafficDataParentPath)
env.selectMode(mode="train", type="data")

Environment Configuration
Number of Users:        8
Window Length:          200
Dataflow:               thumb_fr
Sigmoid K List:         [0.1, 0.2, 0.3, 0.4, 0.5]
Sigmoid S List:         [10.0, 10.0, 10.0, 10.0, 10.0]
Resource Bar:           5
Bandwidth:              100
Sub Agents:             [[0, 0]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7]]


In [5]:
with open(f'../../../data/processed/offline_dataset/subOptimalAgent_encConfig{configIdx}_{envParams["sub_agents_idx"]}.pkl', 'rb') as f:
    dataset_expert = pickle.load(f)

dataset_off = {
    'observations': dataset_expert['uRecord'],     
    'actions': dataset_expert['actionsRecord'], 
    'rewards': dataset_expert['rewardRecord'], 
    'next_observations': dataset_expert['uNextRecord']
}
print(f"Avg. packet loss rate: {np.mean(dataset_expert['rewardRecord'])}")
print(f"length of dataset: {len(dataset_off['observations'])}")

Avg. packet loss rate: 0.3253380120298584
length of dataset: 10000


In [6]:
hyperparams = {
    'N_diffusion_steps':30,
    'schedule_type': "vp",
    'abs_action_max': 1.0,
    'gamma': 0.99,
    'lr': 5e-3,
    'decay_lr': True,
    'weight_decay': 0.0,
    'num_critics': 12,
    'lcb_coef': 0.15,
    'q_sample_eta': 1.0,
    'weight_entropy_loss': 0.01,
    'weight_q_loss': 1.0,
    'approximate_action': True,
    'ema_tau': 0.001,
    'ema_period': 20,
    'ema_begin_update': 1000,
    'layer_norm': True,
    'grad_clip': 3.0,
    'device': 'cuda',
}

In [12]:
envInterface = EnvInterface(
    envParams, discrete_state=False,
    n_bits_state=2, base_state=200,
    n_bits_action=2, base_action=200,
)

r = np.random.uniform(0, envParams['B'], size=(1, 8))
a = envInterface.preprocess_action(r)
r_ = envInterface.postprocess_action(a)
print(r)
print(a)
print(r_)

u = np.random.uniform(0, envParams['LEN_window'], size=(1, 8))
u_ = envInterface.preprocess_state(u)
print(u)
print(u_)


[[67.1835668  71.03175665 63.10815359 97.86353889 28.24510863 52.99527706
   6.49203226 18.25851893]]
[[ 0.34367134  0.42063513  0.26216307  0.95727078 -0.43509783  0.05990554
  -0.87015935 -0.63482962]]
[[67.1835668  71.03175665 63.10815359 97.86353889 28.24510863 52.99527706
   6.49203226 18.25851893]]
[[169.76345929 153.12844433 139.79266297  98.55408422 167.81562454
  104.18625132 169.78508995  16.01407888]]
[[ 0.69763459  0.53128444  0.39792663 -0.01445916  0.67815625  0.04186251
   0.6978509  -0.83985921]]
